In [1]:
import sys
import cv2
from typing import List
from acquisition.source_camera_from_video_dataset import SourceCameraVideoDataset
from segmentation import Segmentation
import numpy as np


In [2]:
camera = SourceCameraVideoDataset("/home/cristiano/dataset.mp4")

segmentation = Segmentation()

bpath = "/home/cristiano/mestrado/vision-module/neuralnet"

segmentation.setupNetwork(
    f"{bpath}/rtkbosque.onnx",
    f"{bpath}/rtkbosque_classes.txt",
    f"{bpath}/rtkbosque_colors.txt",
    353,
    288,
    "input",
    "output"
)

In [3]:
frame = camera.capture()
segmentation.segment(frame)

In [9]:
class BEVGenerator_1920x:

    def __init__(self, grid_dims=(500, 700), edge_size=1e-2):
        self.R = np.array([[ 0.99792479,  0.02269552, -0.06025794],
                           [-0.03158676,  0.98803323, -0.15097222],
                           [ 0.05611045,  0.15256228,  0.98669974]])

        self.tvec = np.array([[0.023667  ],
                              [0.34899726],
                              [0.91836986]])

        self.mtx = np.array([[967.24986511,   0.        , 981.31348404],
                             [  0.        , 988.70685899, 728.97850079],
                             [  0.        ,   0.        ,   1.        ]])

        self.dist = np.array([[ 0.00371372, -0.00157878,  0.00621299,  0.00757212,  0.00170795]])

        self.uv_dims = (1440, 1920) # h x w
        self.grid_dims = grid_dims
        self.edge_size = edge_size
        self._compute_uvcoords()

    def _compute_uvcoords(self):
        cam_offset_Z = (-self.tvec[[2]] + 10e-2) / self.edge_size
        cam_offset_X = (-self.tvec[[0]]) / self.edge_size

        og_grid = np.concatenate(
                        np.meshgrid(
                            np.arange(- self.grid_dims[0] // 2 + cam_offset_X, (self.grid_dims[0] + .999) // 2 + cam_offset_X, 1),
                            0 , # I want only the floor information, such that height = 0
                            np.arange(self.grid_dims[1] + cam_offset_Z - 1, cam_offset_Z - .999, -1),
                            1
                            )
                        )

        og_grid = np.moveaxis(og_grid, 0, 2)
        og_grid_3dcoords = og_grid.copy()
        og_grid_3dcoords[:, :, :3, :] *= self.edge_size

        # Transform the world coords to cam coords
        RT = np.eye(4)
        RT[:3, :3] = self.R
        RT[:3, 3] = self.tvec.ravel()
        og_grid_camcoords = (RT @ og_grid_3dcoords.reshape(-1, 4).T)
        og_grid_camcoords = og_grid_camcoords.T.reshape(self.grid_dims + (4,))
        og_grid_camcoords /= og_grid_camcoords[..., [2]]
        og_grid_camcoords = og_grid_camcoords[..., :3]

        # Transform the cam coords to image coords
        og_grid_uvcoords = (self.mtx @ og_grid_camcoords.reshape(-1, 3).T)
        og_grid_uvcoords = og_grid_uvcoords.T.reshape(self.grid_dims + (3,))
        og_grid_uvcoords = og_grid_uvcoords.round().astype(int)
        og_grid_uvcoords = og_grid_uvcoords[..., :2]
        og_grid_uvcoords = np.moveaxis(og_grid_uvcoords, 0, 1)

        og_grid_uvcoords[..., 1] = og_grid_uvcoords[..., 1].clip(0, self.uv_dims[0])
        og_grid_uvcoords[..., 0] = og_grid_uvcoords[..., 0].clip(0, self.uv_dims[1])
        self.og_grid_uvcoords = og_grid_uvcoords

    def __call__(self, x):
        x = np.pad(x, ((0, 1), (0, 1), (0, 0)), mode='constant', constant_values=0)
        return x[self.og_grid_uvcoords[..., 1], self.og_grid_uvcoords[..., 0]]

In [10]:
bev_generator = BEVGenerator_1920x(grid_dims=(500, 500), edge_size=1e-2)
img = segmentation.getMatrixImage()

cv2.imwrite('original.png',img)



True

In [11]:
bev = bev_generator(img)
cv2.imwrite('bev.png', bev)


IndexError: index 688 is out of bounds for axis 0 with size 289